1. Write a approx 750-800 (3PAGE) word description of: 
    - Chatbot application (e.g., use-case, domain of operation).
    - Atleast 3 different test cases that clearly illustrate chatbot behavior.
    - How you connected your work to lecture material 
    - Advanced techniques that you used

If you are submitting a PDF, please mention that here. 

-----------------------------------------------------
##### PASTE YOUR TEXT HERE
-----------------------------------------------------

------------
2. CODE
-----------

In [9]:
import requests
from requests.structures import CaseInsensitiveDict
from datetime import date

get_access_token = requests.post('https://id.twitch.tv/oauth2/token?client_id=ch28sul1esy1ul6ye36on4r49f6qxr&client_secret=b0yv81ftmbt2isestd8slt4n7qfd5r&grant_type=client_credentials')
access_token = get_access_token.json()['access_token']

url = "https://api.igdb.com/v4/games"

user_genre = 'Platform'
user_platform = 'PlayStation 2'
user_vibes = ['bandicoot', 'spinning', 'crates', 'wumpa', 'collectibles', 'checkpoints']



headers = CaseInsensitiveDict()
headers["Client-ID"] = "ch28sul1esy1ul6ye36on4r49f6qxr"
headers["Authorization"] = "Bearer " + access_token
headers["grant_type"] = "client_credentials"
headers["Content-Type"] = "application/json"
body = f'''
fields name, genres.name, platforms.name, release_dates.date, rating, summary, keywords.name;
where platforms.name = "{user_platform}" & genres.name = "{user_genre}";
limit 40;
'''
response = requests.post(url, headers=headers, data=body.encode('utf-8'))
data = response.json()

for game in data:
    game_word_set = set()
    #gather all fields to check for keywords in
    game_word_set = set()
    if 'keywords' in game:
        for keyword in game['keywords']:
            game_word_set.add(keyword['name'])
    if 'summary' in game:
        game_word_set.add(game['summary'])
    for vibe in user_vibes:
        if vibe in game_word_set:
            game['keyword_count'] = game.get('keyword_count', 0) + 1
    if('keyword_count' in game):
        print(game['name'] + " contains " + str(game['keyword_count']) + " keywords: ")
        
    else:
        print(game['name'] + " contains no keywords")


games_with_keywords = [game for game in data if 'keyword_count' in game]
sorted_games_with_keywords = sorted(games_with_keywords, key=lambda x: x['keyword_count'], reverse=True)


games_without_keywords = [game for game in data if 'keyword_count' not in game]
sorted_games_without_keywords = sorted(games_without_keywords, key=lambda x: x.get('rating', 0), reverse=True)

sorted_games = sorted_games_with_keywords + sorted_games_without_keywords   

for game in sorted_games:
    result = game['name']
    if 'keyword_count' in game:
        result += ", Keywords matched: " + str(game['keyword_count'])
    if 'rating' in game:
        result += ", Rating: " + str(game['rating'])
    print(result)
    print('\n')

    if 'release_dates' in game:
        if 'date' in game['release_dates'][0]:
            print(date.fromtimestamp(game['release_dates'][0]['date']).year)
    if 'summary' in game:
        print('Summary: ' + game['summary'])
        print('\n')



LEGO Indiana Jones: The Original Adventures contains 1 keywords: 
MDK2 contains no keywords
God of War II contains no keywords
Prince of Persia: The Sands of Time contains 1 keywords: 
Prince of Persia: Warrior Within contains no keywords
Oddworld: Munch's Oddysee contains no keywords
Haunted Castle contains no keywords
Tomb Raider: Anniversary contains 1 keywords: 
Tomb Raider: Underworld contains 1 keywords: 
Crash Bandicoot: The Wrath of Cortex contains no keywords
Crash Twinsanity contains 3 keywords: 
Crash of the Titans contains 2 keywords: 
Crash: Mind Over Mutant contains no keywords
Crash Tag Team Racing contains no keywords
Ratchet & Clank contains 2 keywords: 
Ty the Tasmanian Tiger contains no keywords
Ty the Tasmanian Tiger 2: Bush Rescue contains no keywords
Ty the Tasmanian Tiger 3: Night of the Quinkan contains no keywords
Psychonauts contains 1 keywords: 
Metal Slug 5 contains 1 keywords: 
Metal Slug 6 contains no keywords
Jak and Daxter: The Precursor Legacy contains 

In [ ]:
from random import Random
import re
import json
import nltk
import requests
from requests.structures import CaseInsensitiveDict
from datetime import date 
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')


def load_json():
    with open("chatbot.json") as file:
        data = json.load(file)
    return data

def match_pattern_to_intent(user_input):
    intent = None
    for pattern_to_intent in data["pattern_to_intent"]:
        if re.search(pattern_to_intent["pattern"], user_input):
            intent = pattern_to_intent["intent"]
            break
    return intent
def match_intent_to_response(intent):   
    response = None
    if intent in data["intent_to_response"]:
        responseList = data["intent_to_response"][intent]
        response = responseList[Random().randint(0, len(responseList) - 1)]
    return response
def match_intent_to_action(intent):
    action = None
    if intent in data["intent_to_action"]:
        action = data["intent_to_action"][intent]
    return action
def get_user_confirmation(prompt):
    print("Frank: " + prompt)
    user_input = input("You: ").lower()
    if match_pattern_to_intent(user_input) == "positive":
        return True
    elif match_pattern_to_intent(user_input) == "negative":
        return False
    else:
        return None
def interaction_loop():
    print("Frank: Hello! My name is Frank! How can I help?")
    while True:
        user_input = input("You: ").lower()
        intent = match_pattern_to_intent(user_input)
        action = match_intent_to_action(intent)
        if intent is not None:
            print_response_after_action = True
            if action and action in globals():
                print_response_after_action = globals()[action](user_input)
            if print_response_after_action:
                response = match_intent_to_response(intent)
                if response is not None:
                    print("Frank: " + response.format(**memories)) 
        else:
            print("Frank: " + "I'm sorry, I don't understand that. Can you please rephrase?")
#INTENT FUNCTIONS
def remember_name(input):
    user_name_re = r"my name is|I am|I'm|call me"
    user_name = input[re.search(user_name_re, input).start():] #remove string before regex recongized
    user_name = re.sub(user_name_re, "", user_name).split()[0] #remove regex and split by space, first word is name
    if memories.get("user_name") and memories["user_name"].lower() != user_name:
        if not get_user_confirmation("Frank: Hold up, I thought your name was {}, would you like to change this?".format(memories["user_name"])):
            print("Frank: Okay, I'll stick with that.")
            return False
    memories.update({"user_name": user_name.capitalize()})
    return True


def remember_keywords(user_input):
    custom_stopwords = set(nltk.corpus.stopwords.words('english')).union({"game", "games", "like", "something"})
    user_input = remove_all_patterns(user_input)
    if memories.get("keywords"):
        if not get_user_confirmation("Frank: Sure, I had you down for games with the following vibes: {} Should I update this?".format(", ".join(memories["keywords"]) + ".".capitalize())):
            print("Frank: Okay, I'll stick with that.")
            return False
    tokens = nltk.word_tokenize(user_input)
    tagged = nltk.pos_tag(tokens)
    keywords = [word for word, pos in tagged if pos in ['JJ', 'JJR', 'JJS', 'NN', 'NNP', 'NNS', 'VB', 'VBG'] and word not in custom_stopwords]
    memories.update({"keywords": keywords})
    return True


def remember_platform(user_input):
    platform = None
    for pattern_to_platform in data["pattern_to_platform"]:
        if re.search(pattern_to_platform["pattern"], user_input):
            platform = pattern_to_platform["platform"]
            break
    if platform is not None:
        if memories.get("platform") and memories["platform"] != platform:
            if not get_user_confirmation("Frank: I remember you said you own a {}, would you like to update this?".format(memories["platform"])):
                print("Frank: Okay, I'll stick with that.")
                return False
        memories.update({"platform": platform})
    return True

def remember_genre(user_input):
    genre = None
    for pattern_to_genre in data["pattern_to_genre"]:
        if re.search(pattern_to_genre["pattern"], user_input):
            genre = pattern_to_genre["genre"]
            break
    if genre is not None:
        if memories.get("genre") and memories["genre"] != genre:
            if not get_user_confirmation("Frank: You mentioned you were into {} previously. Should I update this?".format(memories["genre"])):
                print("Frank: Okay, I'll stick with that.")
                return False
        memories.update({"genre": genre})
    return True
    
def recommend_game(user_input):
    remember_platform(user_input)
    remember_genre(user_input)
    remember_keywords(user_input)
    if not memories.get("platform"):
        print("Frank: I'm sorry, I don't know what platform you're on. Can you tell me?")
        remember_platform(input("You: "))
    if not memories.get("genre"):
        print("Frank: I'm sorry, I don't know what genre you're into. Can you tell me?")
        remember_genre(input("You: "))
    get_access_token = requests.post('https://id.twitch.tv/oauth2/token?client_id=ch28sul1esy1ul6ye36on4r49f6qxr&client_secret=b0yv81ftmbt2isestd8slt4n7qfd5r&grant_type=client_credentials')
    access_token = get_access_token.json()['access_token']

    url = "https://api.igdb.com/v4/games"

    user_genre = memories.get("genre")
    user_platform = memories.get("platform")
    user_keywords = memories.get("keywords")

    headers = CaseInsensitiveDict()
    headers["Client-ID"] = "ch28sul1esy1ul6ye36on4r49f6qxr"
    headers["Authorization"] = "Bearer " + access_token
    headers["grant_type"] = "client_credentials"
    headers["Content-Type"] = "application/json"
    body = f'''
    fields name, genres.name, platforms.name, release_dates.date, rating, summary, keywords.name, themes.name;
    where platforms.name = "{user_platform}" & genres.name = "{user_genre}";
    limit 500;
    '''
    response = requests.post(url, headers=headers, data=body.encode('utf-8'))
    response = response.json()
    response = [game for game in response if game['name'] not in memories.get('recommended_games', set())]
    
    
    for game in response:
        game.update({'keyword_count': get_keyword_count_for_game(game, user_keywords)})

    games_with_keywords = [game for game in response if 'keyword_count' in game]
    sorted_games_with_keywords = sorted(games_with_keywords, key=lambda x: x['keyword_count'], reverse=True)

    games_without_keywords = [game for game in response if 'keyword_count' not in game]
    sorted_games_without_keywords = sorted(games_without_keywords, key=lambda x: x.get('rating', 0), reverse=True)

    sorted_games = sorted_games_with_keywords + sorted_games_without_keywords   

    print("Frank: I've got some for you, take a look! Say yes if you're interested, or no if you're not.")

    response_list = data["intent_to_response"]["recommend"]
    for game in sorted_games:

        result = game['name']
        if 'keyword_count' in game:
            result += ", Keywords matched: " + str(game['keyword_count'])
        if 'rating' in game:
            result += ", Rating: " + str(game['rating'])

        remember_previously_recommended_title(game['name'])
        if not get_user_confirmation(response_list[Random().randint(0, len(response_list) - 1)].format(name=game['name'])):
            continue
        else:
            describe_game(game)
            break
    return False

def get_keyword_count_for_game(game, keywords):
    keyword_counter = 0
    game_word_set = set()
    if 'keywords' in game:
        for keyword in game['keywords']:
            game_word_set.add(keyword['name'])
    if 'summary' in game:
        game_word_set.add(game['summary'])
    if 'themes' in game:
        for theme in game['themes']:
            game_word_set.add(theme['name'])

    for keyword in keywords:
        if keyword in game_word_set:
            keyword_counter += 1
    return keyword_counter

def remember_previously_recommended_title(game_name):
    if 'recommended_games' not in memories:
        memories['recommended_games'] = set()
    if game_name not in memories['recommended_games']:
        memories['recommended_games'].add(game_name)

def describe_game(game):
    result = game['name']
    if 'release_dates' in game:
        if 'date' in game['release_dates'][0]:
            result += " ("+str(date.fromtimestamp(game['release_dates'][0]['date']).year)+")"
    if 'summary' in game:
        result += ": " + game['summary']    
    print(result)

def remove_all_patterns(input):
    for pattern in data["pattern_to_intent"] + data["pattern_to_platform"] + data["pattern_to_genre"]:
        input = re.sub(pattern["pattern"], "", input)
    return input

    
    
memories = {}
data = load_json()
interaction_loop()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\megaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Frank: Hello! My name is Frank! How can I help?


You:  recommend me ps1 arcade game set in japan


Frank: I've got some for you, take a look! Say yes if you're interested, or no if you're not.
Frank: How about Incredible Crisis?


In [5]:
# ATLEAST 3 INTERESTING RESULTS - AND FUNCTION CALLS TO GENERATE THEM


3. Reflect on project management (approx 250 words) 
    - Timeliness: Reflect on how consistently you made an effort meet deadlines
    - Organization: Reflect on how managed the code-base as the project size increased

-----------------------------------------------------

Reflection goes here

-----------------------------------------------------


4. Process reflection (approx 200 words) - Discuss the week by week iterative development of your chatbot.
    - Describe each week of chatbot devleopment.
    - What was the feedback you received? How did you work on the feedback to improve chatbot? What new features did you add? 

-----------------------------------------------------

Reflection goes here

-----------------------------------------------------
